In [ ]:
!pip install datasets evaluate transformers[sentencepiece] torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("csv", data_files='/content/drive/MyDrive/nlp-getting-started/pushkin_or_not.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a4059f4af876573e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

checkpoint = "sberbank-ai/ruRoberta-large"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
import torch
from datasets import DatasetDict

def tokenization(raw):
    result = raw
    result['input_ids'] = torch.tensor(tokenizer(raw["text"])["input_ids"])
    result['attention_mask'] = torch.tensor(tokenizer(raw["text"])["attention_mask"])
    return result

tokenized_train = train_dataset["train"].map(tokenization)

dataset = DatasetDict({'train': tokenized_train})
dataset = dataset['train'].train_test_split()

  0%|          | 0/28563 [00:00<?, ?ex/s]

In [ ]:
from transformers import DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="test_trainer",
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    eval_steps=10,
    logging_steps=2,
    num_train_epochs=6,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=10,
    gradient_accumulation_steps=256,
    warmup_steps=5,
    weight_decay=0.1,
)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at sberbank-ai/ruRoberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [ ]:
import sklearn

df = tokenized_train.to_pandas()
classes_zero = df[df['label'] == 0]
classes_one = df[df['label'] == 1]

zero_numpy = classes_zero['label'].to_numpy()
one_numpy = classes_one['label'].to_numpy()
all_together = np.concatenate((zero_numpy, one_numpy))
unique_classes = np.unique(all_together)

balanced_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',
                                                                   classes=unique_classes,
                                                                   y=all_together).tolist()
print(balanced_weights)

[0.7559949182150231, 1.4765818858560793]


In [ ]:
from torch import nn
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(balanced_weights).to(device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, author. If text, author are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 21422
  Num Epochs = 6
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 256
  Total optimization steps = 246
  Number of trainable parameters = 355361794
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method i

Step,Training Loss,Validation Loss,Accuracy
10,0.648700,0.683533,0.672175
20,0.603600,0.583244,0.673575
30,0.546700,0.563702,0.753256
40,0.518100,0.511477,0.693600
50,0.461400,0.489098,0.793166
60,0.490900,0.449019,0.781123
70,0.517900,0.498548,0.807310
80,0.426400,0.451601,0.748495
90,0.365100,0.460164,0.808990
100,0.355500,0.430993,0.803109


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, author. If text, author are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7141
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-10
Configuration saved in test_trainer/checkpoint-10/config.json
Model weights saved in test_trainer/checkpoint-10/pytorch_model.bin
tokenizer config file saved in test_trainer/checkpoint-10/tokenizer_config.json
Special tokens file saved in test_trainer/checkpoint-10/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, author. If text, author are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *

In [ ]:
from transformers import AutoModelForSequenceClassification

# The checkpoint with the least validation lost is taken
path = 'test_trainer/checkpoint-120'
result_model = AutoModelForSequenceClassification.from_pretrained(path, num_labels=2)

loading configuration file test_trainer/checkpoint-120/config.json
Model config RobertaConfig {
  "_name_or_path": "test_trainer/checkpoint-120",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file test_trainer/checkpoint-120/pytorch_model.bin
All model checkpoint weights were used when initiali

In [ ]:
metric = evaluate.load("f1")

def compute_metrics_f1(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [ ]:
trainer_f1 = Trainer(
    result_model,
    training_args,
    #train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_f1,
)

In [ ]:
trainer_f1.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, author. If text, author are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7141
  Batch size = 8


{'eval_loss': 0.4044986963272095,
 'eval_f1': 0.804210785607894,
 'eval_runtime': 87.1522,
 'eval_samples_per_second': 81.937,
 'eval_steps_per_second': 10.246}

In [ ]:
from sklearn.metrics import f1_score
from numpy import random

# f1 for all ones
df = dataset["train"].to_pandas()
y_true = df["label"].tolist()
y_ones = [1] * len(df)

score_ones = f1_score(y_true=y_true, y_pred=y_ones, average='binary')
print(f'f1 for all ones: {score_ones}')

# f1 for random results
y_random = random.choice([0, 1], size = (len(df)))
score_random = f1_score(y_true=y_true, y_pred=y_random, average='binary')
print(f'f1 for all random: {score_random}')

f1 for all ones: 0.5057719806089352
f1 for all random: 0.4018300507727501


In [ ]:
from transformers import pipeline

classif = pipeline("text-classification", model=result_model, tokenizer=tokenizer, device=0)
result = classif("через несколько минут при огне конек как тут")
print(result)

[{'label': 'LABEL_0', 'score': 0.6440086364746094}]


In [ ]:
gorbunok = load_dataset("csv", data_files='/content/drive/MyDrive/nlp-getting-started/gorbunok.csv')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-385d0678fd5e8544/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def gorbunok_prediction(raw):
  classif = pipeline("text-classification", model=result_model, tokenizer=tokenizer, device=0)
  result = raw
  prediction = classif(raw["Text"])
  result["result"] = "Not Pushkin" if prediction[0]['label'] == 'LABEL_0' else "Pushkin"
  result["score"] = prediction[0]['score']
  return result

gorbunok_predicted = gorbunok["train"].map(gorbunok_prediction)

  0%|          | 0/2425 [00:00<?, ?ex/s]

In [ ]:
df_kg = gorbunok_predicted.to_pandas()
print(df_kg["result"].value_counts())

Not Pushkin    1841
Pushkin         584
Name: result, dtype: int64


In [ ]:
print(df_kg.groupby("result")["score"].mean())

result
Not Pushkin    0.740185
Pushkin        0.643900
Name: score, dtype: float64
